# Generate few shot splits

In [1]:
import json
import random
import os
from PIL import Image
from itertools import product
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from collections import Counter

from domain import SYM2PROG, Program, NULL_VALUE, FEWSHOT_OPERATORS, FEWSHOT_NUMBERS
import sys
from helper import *

In [2]:
split2sym = {}
for split in splits:
    sym_set = {k:Iterator(v) for k,v in json.load(open('sym_%s.json'%split)).items()}
    split2sym[split] = sym_set

In [3]:
fewshot_nums = FEWSHOT_NUMBERS
fewshot_ops = FEWSHOT_OPERATORS
fewshot_concepts = fewshot_nums + fewshot_ops
op2precedence.update({'a': op2precedence['+'], 'b': op2precedence['+'], 'c': op2precedence['*'], 'd': op2precedence['*']})
sym2arity.update({x: 0 for x in fewshot_nums})
sym2arity.update({x: 2 for x in fewshot_ops})
functions = [lambda: 11, lambda: 12,
             lambda x,y: max(x, y), lambda x,y: min(x, y), 
             lambda x,y: math.ceil((x+y)/2), lambda x,y: math.ceil(2*x*y/(x+y)) if x+y != 0 else None]
programs = [Program(f) for f in functions]
SYM2PROG.update(dict(zip(fewshot_concepts, programs)))

In [4]:
min_num = 0
max_num = 10
num_list = list(map(str, range(min_num, max_num)))
op_list = ['+', '-', '*', '/']
lps = '('
rps = ')'

def enumerate_expression(n_op):
    if n_op == 0:
        return [[x] for x in num_list]
    
    expressions = []
    
    arity = 1
    ops = [op for op in op_list if sym2arity[op] == arity]
    inputs = enumerate_expression(n_op-1)
    expressions.extend(product(ops, inputs))
    
    arity = 2
    ops = [op for op in op_list if sym2arity[op] == arity]
    inputs = []
    for i in range(n_op):
        input_1 = enumerate_expression(i)
        input_2 = enumerate_expression(n_op - 1 - i)
        inputs.extend(product(input_1, input_2))
    expressions.extend(product(ops, inputs))
    
    expressions = [flatten(x) for x in expressions]
    return expressions

def sample_one_expr(n_op):
    if n_op == 0:
        return [random.choice(num_list)]
    op = random.choice(op_list)
    if sym2arity[op] == 1:
        return [op] + sample_one_expr(n_op-1)
    else:
        i = random.randint(0, n_op-1)
        j = n_op - 1 - i
        return [op] + sample_one_expr(i) + sample_one_expr(j)

def sample_expression(sym, n_op, n_instances, min_value=0, max_value=float('inf'), res_max_ratio=1.0):
    max_ins_res = int(n_instances * res_max_ratio)
    res2n_ins = {}
    expressions = []
    with tqdm(total=n_instances) as pbar:
        while len(expressions) < n_instances:
            expr = sample_one_expr(n_op)
            if expr in expressions:
                continue
            head = parse_prefix(expr)
            res, res_all = eval_expr(expr, head)
            if not res:
                continue
            if res not in res2n_ins:
                res2n_ins[res] = 0
            if res2n_ins[res] >= max_ins_res:
                continue
            
            max_res = max([x for x in res_all if x is not None])
            if max_res >= min_value and max_res <= max_value and sym in expr:
                expressions.append(expr)
                res2n_ins[res] += 1
                pbar.update(1)
    return expressions

def generate_expression(sym, n_op, n_instances, min_value=0, max_value=float('inf'), res_max_ratio=1.0):
    if n_op <= 2:
        expressions = []
        for expr in enumerate_expression(n_op):
            head = parse_prefix(expr)
            res, res_all = eval_expr(expr, head)
            if res is not None:
                max_res = max([x for x in res_all if x is not None])
                if max_res >= min_value and max_res <= max_value and sym in expr:
                    expressions.append(expr)
    else:
        assert n_instances is not None
        expressions = sample_expression(sym, n_op, n_instances, min_value, max_value, res_max_ratio)
    random.shuffle(expressions)
    expressions = expressions[:n_instances]
    
    temp = []
    for expr in expressions:
        expr = prefix2infix(expr)
        head = parse_infix(expr)
        res, res_all = eval_expr(expr, head)
        if res is not None:
            temp.append((expr, head, res, res_all))
    
    return temp

In [5]:
fs2dataset = {}
for i, fs in enumerate(fewshot_concepts):
    random.seed(12306)
    max_op = 20
    max_op_train = 10
    max_value_train = 100
    max_value_test = 10000
    n_train = 100
    n_test = 100
    n_val = 10
    res_max_ratio = 0.05
    train_exprs = []
    val_exprs = []
    test_exprs = []
    
    if fs in fewshot_nums:
        num_list.append(fs)
    if fs in fewshot_ops:
        op_list.append(fs)
    print(num_list)
    print(op_list)
    
    # generate symbolic expression
    for n_op in range(max_op + 1):
        train_exprs_i = []
        val_exprs_i = []
        test_exprs_i = []
        
        if not (fs in fewshot_ops and n_op == 0):
            if n_op > max_op_train:
                n_train = 0
            expressions = generate_expression(fs, n_op, n_train + n_val + n_test, 
                                        max_value=max_value_train, res_max_ratio=res_max_ratio)
            if len(expressions) < n_train:
                print('there are not enough expressions for train set. Repeat them.')
                expressions = expressions * (n_train // len(expressions) + 1)
                expressions = expressions[:n_train]



            # split 'I'
            if n_train > 0:
                train_exprs_i.extend(expressions[:n_train])
                val_exprs_i.extend(expressions[:min(n_train, n_val)])
                test_exprs_i.extend(expressions[:min(n_train, n_test)])

            # split 'SS' or 'LS'
            val_exprs_i.extend(expressions[n_train:n_train+n_val])
            test_exprs_i.extend(expressions[n_train+n_val:])

            # split 'SL' or 'LL'
            expressions = generate_expression(fs, n_op, n_val + n_test,
                            min_value=max_value_train+1, max_value=max_value_test, res_max_ratio=res_max_ratio)
            val_exprs_i.extend(expressions[:n_val])
            test_exprs_i.extend(expressions[n_val:n_val+n_test])

        print(n_op, len(train_exprs_i), len(val_exprs_i), len(test_exprs_i))
        train_exprs.append(train_exprs_i)
        val_exprs.append(val_exprs_i)
        test_exprs.append(test_exprs_i)

    split2exprs = {'train': train_exprs, 'val': val_exprs, 'test': test_exprs}
    print([(k, sum([len(x) for x in v])) for k, v in split2exprs.items()])
    
    split2dataset = {}
    for split in splits:
        sym_set = split2sym[split]
        exprs = split2exprs[split]
        dataset = []
        idx = 0
        for n_op, expr_list in enumerate(exprs):
            for e in expr_list:
                if split in ['val', 'test']:
                    if e[0] in train_exprs:
                        evaluation = 'I'
                    else:
                        max_value = max([x for x in e[3] if x is not None])
                        evaluation = 'S' if n_op <= max_op_train else 'L'
                        evaluation += 'S' if max_value <= max_value_train else 'L'

                img_paths = generate_img_paths(e[0], sym_set)
                sample = {'id': '%s_%08d'%(split, idx), 'img_paths':img_paths, 
                      'expr': e[0], 'head': e[1], 'res': e[2], 'res_all': e[3]}
                if split in ['val', 'test']:
                    sample['eval'] = evaluation
                idx += 1
                dataset.append(sample)
        split2dataset[split] = dataset
        if split == 'train':
            train_exprs = {x['expr'] for x in dataset}
        print(split, len(dataset))
    
    counts = sorted(Counter([x['eval'] for x in split2dataset['test']]).items())
    total_count = sum([c for _, c in counts])
    counts = [(k, v, round(v/total_count*100, 2)) for k, v in counts]
    print(counts)
    
    if fs in fewshot_nums:
        num_list.remove(fs)
    if fs in fewshot_ops:
        op_list.remove(fs)
    fs2dataset[fs] = split2dataset

json.dump(fs2dataset, open('fewshot_dataset.json', 'w'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'x']
['+', '-', '*', '/']
there are not enough expressions for train set. Repeat them.
0 100 10 100
there are not enough expressions for train set. Repeat them.
1 100 11 100


 68%|███████████████████████████▉             | 75/110 [00:00<00:00, 735.74it/s]

2 100 30 300


 77%|███████████████████████████████▋         | 85/110 [00:00<00:00, 844.36it/s]

3 100 30 300


...

18 0 20 200



 58%|███████████████████████▊                 | 64/110 [00:00<00:00, 619.69it/s]

19 0 20 200


100%|████████████████████████████████████████| 110/110 [00:00<00:00, 370.57it/s]


20 0 20 200
[('train', 1000), ('val', 470), ('test', 4700)]
train 1000
val 470
test 4700
[('I', 1000, 21.28), ('LL', 1000, 21.28), ('LS', 1000, 21.28), ('SL', 800, 17.02), ('SS', 900, 19.15)]
